<a href="https://colab.research.google.com/github/awal015/CapstoneStep4/blob/main/How_to_find_new_Board_Games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
threnjen_board_games_database_from_boardgamegeek_path = kagglehub.dataset_download('threnjen/board-games-database-from-boardgamegeek')

print('Data source import complete.')


# Goal

The goal of this notebook is to explore two ways of finding new Board Games you might like.

The first one is the Community approach, where you will have the possibility to restrict the
* Amount of Players,
* Time played
* and Number of User Ratings

Finally, you will declare a **SucessFactor** in
* Number of Users owning the Game,
* the Average User Rating
* or the Difficulty of the Game,

and be able to go through the top 15 Board Games within your declared boundaries.

The second approach is based on User Ratings. Your input will be some of your favorite Board Games and you will be presented with other Board Games that likeminded users also loved, allowing similar restriction as before. This approach will be split into two subideas.

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None) #Always show all columns
pd.set_option('display.max_rows', 200) #Always show upto 200 rows

# Data Preparation

In a first step, we must load the data and change some of the columns values like **Cat:War** to **CatWar**, because they are subpar to work with.

In [ ]:
df = pd.read_csv('/kaggle/input/board-games-database-from-boardgamegeek/games.csv')

df.columns = ['BGGId', 'Name', 'Description', 'YearPublished', 'GameWeight',
       'AvgRating', 'BayesAvgRating', 'StdDev', 'MinPlayers', 'MaxPlayers',
       'ComAgeRec', 'LanguageEase', 'BestPlayers', 'GoodPlayers', 'NumOwned',
       'NumWant', 'NumWish', 'NumWeightVotes', 'MfgPlaytime', 'ComMinPlaytime',
       'ComMaxPlaytime', 'MfgAgeRec', 'NumUserRatings', 'NumComments',
       'NumAlternates', 'NumExpansions', 'NumImplementations',
       'IsReimplementation', 'Family', 'Kickstarted', 'ImagePath',
       'RankBoardgame', 'RankStrategygames', 'RankAbstracts',
       'RankFamilygames', 'RankThematic', 'RankCgs', 'RankWargames',
       'RankPartygames', 'RankChildrensgames', 'CatThematic',
       'CatStrategy', 'CatWar', 'CatFamily', 'CatCGS', 'CatAbstract',
       'CatParty', 'CatChildrens']

## Create Subdataframe

Secondly, we exclude some factors we do not care about like **NumExpansions, NumImplementations** etc. and create a subdataframe based on all the other factors.

In [ ]:
sub_df = df.drop(['NumAlternates', 'NumExpansions', 'NumImplementations',
       'IsReimplementation', 'Family', 'Kickstarted', 'ImagePath','CatThematic',
       'CatStrategy', 'CatWar', 'CatFamily', 'CatCGS', 'CatAbstract',
       'CatParty', 'CatChildrens', 'LanguageEase', 'BayesAvgRating', 'NumComments',
        'RankFamilygames', 'RankThematic', 'RankChildrensgames', 'YearPublished', 'NumWant',
       'NumWish', 'ComMinPlaytime', 'ComAgeRec', 'MfgAgeRec',
       'RankBoardgame', 'RankStrategygames', 'RankAbstracts', 'RankCgs',
       'RankWargames', 'RankPartygames', 'Description'], axis = 1)

# Extracting Information for Board Games

Before starting to suggest New Board Games, we want to make sure that we extract all the information about each game, so that we can present a full picture when we start with the suggestions.

In this section, we will add Information about
* **Category**
* **Subcategories**
* **Themes**

to the *sub_df*.

## Extracting the Information from the One Hot Columns

A lot of Information is encoded as a One Hot Columns, like e.g. the **Categories**. To decode them, the following function will be used.

### From One Hot to Value

In [ ]:
def from_one_hot_to_category(row, names_array): #take a row of a dataframe and a array of names
    cat_array = np.array(row) #interpret the row as an np.array
    names_array = [" " + s + "," for s in names_array] #Add an " " at the beginning and a "," at the end of every name in names_array
    return ''.join(np.repeat(names_array, cat_array)) #Output names_array * cat_array.transpose, whereby string * int = string if int = 1 and 0 else

### Category

In [ ]:
df['Category'] = df[['CatThematic',
       'CatStrategy', 'CatWar', 'CatFamily', 'CatCGS', 'CatAbstract',
       'CatParty', 'CatChildrens']].apply(lambda row: from_one_hot_to_category(row, list(row.index)), axis = 1) #apply from_one_hot_to_category rowwise

df['Category'] = df['Category'].str[1:-1].str.replace('Cat','') #Get rid of the first " " at left hand side and the last "," at the right hand side

sub_df = sub_df.merge(df[['BGGId', 'Category']], on = 'BGGId', how = 'left') #Add the new column Category to sub_df

### Subcategories

In [ ]:
subcategory = pd.read_csv('/kaggle/input/board-games-database-from-boardgamegeek/subcategories.csv') #Load the data

subcategory['Subcategories'] = subcategory.drop('BGGId', axis = 1).apply(lambda row: from_one_hot_to_category(row, list(row.index)), axis = 1) #apply from_one_hot_to_category rowwise

subcategory['Subcategories'] = subcategory['Subcategories'].str[1:-1] #Get rid of the first " " at left hand side and the last "," at the right hand side

sub_df = sub_df.merge(subcategory[['BGGId', 'Subcategories']], on = 'BGGId', how = 'left') #Add the new column Subcategories to sub_df

### Themes

In [ ]:
themes = pd.read_csv('/kaggle/input/board-games-database-from-boardgamegeek/themes.csv') #Load the data

themes['Themes'] = themes.drop('BGGId', axis=1).apply(lambda row: from_one_hot_to_category(row, list(row.index)), axis = 1) #apply from_one_hot_to_category rowwise

themes['Themes'] = themes['Themes'].str[1:-1] #Get rid of the first " " at left hand side and the last "," at the right hand side

sub_df = sub_df.merge(themes[['BGGId', 'Themes']], on = 'BGGId', how = 'left') #Add the new column Themes to sub_df

## Reordering sub_df and loading user_ratings Dataframe

Next, we want to reorder *sub_df* and load the User Ratings into pandas.

In [ ]:
sub_df = sub_df[['Name', 'AvgRating', 'StdDev', 'NumUserRatings', 'Category', 'Subcategories', 'Themes', 'ComMaxPlaytime', 'MinPlayers',
       'MaxPlayers', 'BestPlayers', 'GoodPlayers', 'GameWeight', 'MfgPlaytime', 'NumOwned', 'BGGId']] #Changing the column order

df_suggestions = sub_df.copy() #creating a seperat dataframe for the second suggestions

user_ratings = pd.read_csv('/kaggle/input/board-games-database-from-boardgamegeek/user_ratings.csv') #Loading the Userratings

user_ratings.isna().sum() / user_ratings.shape[0]

There are very few Usernames missing in the *user_ratings* Dataframe.

We will exclude those.

In [ ]:
user_ratings = user_ratings[~user_ratings.Username.isna()] #Exclude all rows with missing usernames

# Finding new Games

Finally, we get to the heart of the notebook, where one can adjust parameters to get suggestions, one might be interested in.

Remember, there are three factors to adjust:
* **MaxPlaytime:** The highest tolerable number of minutes the game should last;
* **AtleastPlayers:** The lowest number of players that must be allowed to play;
* **NumUserRatings:** The lowest number of User Ratings a game must have on Board Game Geek.

## Community Approach

This subsection is about finding games, based on your parameters, that are most loved by the community.

You may also choose the **SuccessFactor** out of
* **AvgRating:** The Average Rating on BGG (this is the default);
* **NumOwned:** Number of User on BGG owning this game;
* **GameWeight:** Difficulty of the game, according to BGG Users.

In [ ]:
def most_loved(NumUserRatings = 10, MaxPlaytime = 10000,
               AtleastPlayers = 2, SuccessFactor = 'AvgRating'):
    df_pref = sub_df[sub_df.NumUserRatings > NumUserRatings] #Only taking games, where there are more User Ratings than NumUserRatings
    df_pref = df_pref[df_pref.ComMaxPlaytime < MaxPlaytime]  #Only taking games, where the ComMaxPlaytime is atmost MaxPlaytime
    df_pref = df_pref[df_pref.MaxPlayers >= AtleastPlayers]  #Only taking games, where the MaxPlayers is more than or equal to AtleastPlayers
    return df_pref.sort_values(SuccessFactor, ascending = False).head(n=15) #Sort df_pref by the SuccessFactor and return the top 15

In [ ]:
most_loved(NumUserRatings = 100, MaxPlaytime = 180,
           AtleastPlayers = 5, SuccessFactor = 'AvgRating')

## Based on Players who share a passion for our favorite games

In this subsection, you will be presented further suggestions based on likeminded users and the games they love. This will happen in three steps:
1. You input the **BGGId**'s, as a list or int, of your favorite games and other parameters into the function "suggesting_new_games";
2. The function will find users that also loved those games, based on them being in the top 10% of the highest Raters of any or all (if MustLikeAll=True) of those games;
3. It returns the top 15 games based on the **Rating** of those likeminded users.

It is important to note that one must consider tweeking some parameters here. For example, if there is one user in these likeminded users that gave a game a 10-rating, but noone else voted on this game, is this really a game one should suggest?

Accordingly, there are some new parameters in:
* **MinVotes:** This is the least number of likeminded users that need to have voted on a certain game;
* **MustLikeAll:** If this is set to "True", only the users are considered that were in the 90th Percential of all the given **BGGId**'s. Otherwise, it is enough for a user to be in the 90th Percential of one of your games.

In [ ]:
def finding_games(like_minded_users, NumUserRatings = 15, MaxPlaytime = 10000,
                         AtleastPlayers = 0, MinVotes = 10):

    #Find all rows, where the likeminded users voted
    ratings_of_likeminded = user_ratings[user_ratings.Username.isin(like_minded_users)]

    #Find the average Rating and the number of Ratings for every BGGId
    suggestions = ratings_of_likeminded.groupby('BGGId').agg(['mean', 'count'])
    #Exclude all BGGIds, where there were less or equal to MinVotes Ratings
    suggestions = suggestions[suggestions.Rating['count'] > MinVotes]
    #Rename columns
    suggestions.columns = ['MeanRating', 'NumRatings']
    #Sort columns by MeanRating from top to buttom
    suggestions = suggestions.sort_values('MeanRating', ascending = False)
    #Make BGGId go from index to column
    suggestions = suggestions.reset_index()

    #Start excluding rows based on the input parameters in AtleastPlayers, NumUserRatings and MaxPlaytime
    new_games = df_suggestions[df_suggestions.MaxPlayers > AtleastPlayers]
    new_games = new_games[new_games.NumUserRatings > NumUserRatings]
    new_games = new_games[new_games.ComMaxPlaytime <= MaxPlaytime]
    #Attach the NumRatings and MeanRating to new_games based on BGGId
    new_games = new_games.merge(suggestions[['BGGId', 'NumRatings', 'MeanRating']], on = 'BGGId', how = 'left')
    #Get the Top 15 of the new_games Dataframe with respect to MeanRating
    new_games = new_games.sort_values('MeanRating', ascending = False).head(n=15)

    return new_games

In [ ]:
def suggesting_new_games(BGGIds: list or int, NumUserRatings = 15, MaxPlaytime = 10000,
                         AtleastPlayers = 0, MustLikeAll = False, MinVotes = 10):
    if isinstance(BGGIds, list): #If BGGIds is a list
        if MustLikeAll:
            #Initialize user_names
            user_names = set(user_ratings[user_ratings.BGGId == BGGIds[0]].Username.unique())
            #Loop over all BGGIds
            for Id in BGGIds:
                #Intersect the user_names of the 90th Percentile of all your favorite games
                user_names = set(user_ratings[user_ratings.BGGId == Id].Username.unique()).intersection(user_names)
        else:
            #Find all rows, where any of your games were rated
            ratings = user_ratings[user_ratings.BGGId.isin(BGGIds)]
            #Find the 10% of users that liked your games the most
            ratings = ratings[ratings.Rating >= user_ratings.Rating.quantile(0.9)]
            #Go from a dataframe to a list and exclude Usernames if they occur multiple times
            user_names = ratings.Username.unique()
    else:
        #Find all rows, where your game was rated
        ratings = user_ratings[user_ratings.BGGId == BGGIds]
        #Find the 10% of users that liked your game the most
        ratings = ratings[ratings.Rating >= user_ratings.Rating.quantile(0.9)]
        #Go from a dataframe to a list and exclude Usernames if they occur multiple times
        user_names = ratings.Username.unique()

    return finding_games(user_names, NumUserRatings, MaxPlaytime,
                         AtleastPlayers, MinVotes)

In this example, I plugged in "7 Wonders", "Carcassone", "Codenames" and "Scythe".

In [ ]:
new_games = suggesting_new_games([68448, 822, 178900, 169786], MaxPlaytime = 180,
                                 AtleastPlayers = 5, MustLikeAll = True)

new_games

# Finding players with similar interests

Another approach would be to compare our ratings of games with the ratings of other players by defining a distance between players:

$$\sum_{game \in \Omega} (our \; rating(game) - their \; rating(game))^2,$$

where $\Omega$ is the set of games you and the other user have rated. So essentialy we want to find the users, that rated games similarly to us.

After finding likeminded users in this way, we may look again at their favorite games for suggestions.

In this approach, we also incorperate whenever both users did not like a game, an advantadge to the previous approach.

In [ ]:
df[df.BGGId.isin([68448, 822, 178900, 169786, 224517, 84876, 150145, 131357])].Name.unique()

In [ ]:
#Defining our data set with our rating
BGGIds = [68448, 822, 178900, 169786, 224517, 84876, 150145, 131357]
ratings = [7, 6, 8, 8, 9, 8, 9, 6] #The first rating coresponds to the first BGGId
our_ratings = pd.DataFrame(np.array([BGGIds, ratings]).T, columns = ['BGGId', 'Rating'])

In [ ]:
def calculate_distance(row): #Calculate the distance between a users rating and ours
    our_rating = int(our_ratings[our_ratings.BGGId == row.BGGId].Rating)
    return (int(row.Rating) - our_rating)**2

To get better suggestions and reduce some complexity, we will be excluding all users that did not rate at least $min(5, number \; of \; games \; we \; rated)$ of the games we rated.

In [ ]:
def likeminded_users(ratings, NumberOfSimilarUsers = 1000, NumUserRatings = 15, MaxPlaytime = 10000,
                         AtleastPlayers = 0, MustLikeAll = False, MinVotes = 10): #Suggest new games based on likeminded users
    #We only calculate the distance for Board Games that we rated. Therefore all others will be excluded
    sub_ratings = user_ratings[user_ratings.BGGId.isin(our_ratings.BGGId.unique())]

    #Next, we only want to compare to users that rated most of the games we rated
    df_for_valuable_users = pd.DataFrame(sub_ratings.groupby('Username').BGGId.count() >= min(5, our_ratings.shape[0])).reset_index()
    #Here we exclude all users that do not satisfy, having rated at least min(5, number of games we rated)
    users_with_enough_ratings = df_for_valuable_users[df_for_valuable_users.BGGId == True].Username
    #Finally we reduce the size of sub_ratings by excluding users
    sub_ratings = sub_ratings[sub_ratings.Username.isin(users_with_enough_ratings)]

    #Next, we calculate the Distance for all rows in sub_ratings
    sub_ratings['Distance'] = sub_ratings.apply(lambda row: calculate_distance(row), axis = 1)
    #Then we sort the Usernames based on their Distance ascendingly
    smallest_distance = sub_ratings.groupby('Username').Distance.mean().sort_values()
    #Finally we extract the top NumberOfSimilarUsers Usernames
    like_minded_users = smallest_distance.head(n=NumberOfSimilarUsers).index

    #In a final step, we use our suggest_games function to find the games like_minded_users rated highly
    return finding_games(like_minded_users, NumUserRatings, MaxPlaytime,
                         AtleastPlayers, MinVotes)

Finally we present the suggestions.

In [ ]:
likeminded_users(our_ratings, MinVotes = 50)

# Further Ideas

At this point, we have implemented a very simple algorithm to find new Board Games based on other users. A future goal of mine will be to bring Machine Learning and Clustering into the equation to find new relevant factors in how to suggest new Board Games.